In [4]:
import logging
import random
import torch
import numpy as np

logging.basicConfig(level=logging.INFO, format='%(asctime)-15s %(levelname)s: %(message)s')

seed = 666
random.seed(seed)
np.random.seed(seed)

torch.cuda.manual_seed(seed)
torch.manual_seed(seed)

In [7]:
# split data
fold_num = 10
data_file = 'C:/ZhangLI/Codes/DataSet/新闻文本分类/train_set.csv'

import pandas as pd

In [37]:

num = 1000
f = pd.read_csv(data_file, sep='\t', encoding='utf-8', nrows=1000)
f.head()

texts = f['text'].tolist()[:num]
labels = f['label'].tolist()[:num]

# 为什么要转成list呢，是不是encoding

all_texts = []
all_labels = []

# for i in index:
#     pass

for i in range(len(texts)):
    all_texts.append(texts[i])

total = len(labels)
label2id = {}
for i in range(total):
    label = str(labels[i])
    if label not in label2id:
        label2id[label] = [i]
    else:
        label2id[label].append(i)

label2id  # 我怀疑是要进行抽取采样

all_index = [[] for _ in range(fold_num)]

for label, data in label2id.items():
    print('----')
    batch_size = int(len(data) / fold_num)  
    print(batch_size)
    other = len(data) - batch_size * fold_num  
    print(other)
    for i in range(fold_num):  # 0 1 2 3 4 5 6 7 
#         print(i)
        cur_batch_size = batch_size + 1 if i < other else batch_size
        print(cur_batch_size)
        batch_data = [data[i * batch_size + b] for b in range(cur_batch_size)]
        print(batch_data)
        all_index[i].extend(batch_data)

----
15
2
16
[0, 3, 17, 19, 22, 23, 40, 46, 47, 48, 51, 59, 68, 75, 76, 78]
16
[78, 81, 97, 98, 108, 132, 135, 138, 140, 141, 149, 153, 172, 179, 187, 198]
15
[198, 200, 211, 231, 232, 235, 239, 242, 244, 245, 246, 268, 271, 272, 278]
15
[293, 299, 305, 310, 318, 324, 325, 334, 335, 343, 355, 357, 372, 373, 376]
15
[378, 388, 392, 396, 398, 409, 420, 422, 433, 436, 446, 447, 450, 452, 453]
15
[487, 506, 526, 527, 533, 536, 537, 541, 545, 547, 554, 560, 561, 564, 574]
15
[581, 583, 592, 601, 607, 620, 626, 634, 636, 637, 643, 652, 656, 661, 673]
15
[688, 697, 712, 721, 727, 729, 739, 746, 748, 756, 757, 760, 766, 776, 783]
15
[789, 790, 791, 795, 813, 818, 821, 823, 825, 832, 839, 841, 848, 857, 873]
15
[883, 884, 916, 918, 930, 933, 934, 937, 947, 951, 954, 959, 962, 964, 971]
----
1
4
2
[1, 64]
2
[64, 66]
2
[66, 70]
2
[70, 79]
1
[79]
1
[120]
1
[160]
1
[168]
1
[283]
1
[397]
----
11
3
12
[2, 4, 6, 9, 26, 33, 42, 53, 58, 60, 61, 63]
12
[63, 107, 130, 136, 142, 156, 157, 158, 169, 170, 17

In [38]:
# split data to 10 fold


def all_data2fold(fold_num, num=10000):
    fold_data = []
    f = pd.read_csv(data_file, sep='\t', encoding='UTF-8', nrows=num)
    texts = f['text'].tolist()[:num]
    labels = f['label'].tolist()[:num]

    total = len(labels)

    index = list(range(total))
    np.random.shuffle(index)

    all_texts = []
    all_labels = []
    for i in index:
        all_texts.append(texts[i])
        all_labels.append(labels[i])

    label2id = {}
    for i in range(total):
        label = str(all_labels[i])
        if label not in label2id:
            label2id[label] = [i]
        else:
            label2id[label].append(i)

    all_index = [[] for _ in range(fold_num)]
    for label, data in label2id.items():
        # print(label, len(data))
        batch_size = int(len(data) / fold_num)
        other = len(data) - batch_size * fold_num
        for i in range(fold_num):
            cur_batch_size = batch_size + 1 if i < other else batch_size
            # print(cur_batch_size)
            batch_data = [data[i * batch_size + b] for b in range(cur_batch_size)]
            all_index[i].extend(batch_data)

    batch_size = int(total / fold_num)
    other_texts = []
    other_labels = []
    other_num = 0
    start = 0
    for fold in range(fold_num):
        num = len(all_index[fold])
        texts = [all_texts[i] for i in all_index[fold]]
        labels = [all_labels[i] for i in all_index[fold]]

        if num > batch_size:
            fold_texts = texts[:batch_size]
            other_texts.extend(texts[batch_size:])
            fold_labels = labels[:batch_size]
            other_labels.extend(labels[batch_size:])
            other_num += num - batch_size
        elif num < batch_size:
            end = start + batch_size - num
            fold_texts = texts + other_texts[start: end]
            fold_labels = labels + other_labels[start: end]
            start = end
        else:
            fold_texts = texts
            fold_labels = labels

        assert batch_size == len(fold_labels)

        # shuffle
        index = list(range(batch_size))
        np.random.shuffle(index)

        shuffle_fold_texts = []
        shuffle_fold_labels = []
        for i in index:
            shuffle_fold_texts.append(fold_texts[i])
            shuffle_fold_labels.append(fold_labels[i])

        data = {'label': shuffle_fold_labels, 'text': shuffle_fold_texts}
        fold_data.append(data)

    logging.info("Fold lens %s", str([len(data['label']) for data in fold_data]))

    return fold_data


fold_data = all_data2fold(10)

2021-08-12 16:46:41,619 INFO: Fold lens [1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000]


In [40]:
# build train data for word2vec
fold_id = 9

train_texts = []
for i in range(0, fold_id):
    data = fold_data[i]
    train_texts.extend(data['text'])
    
logging.info('Total %d docs.' % len(train_texts))

2021-08-12 16:47:36,199 INFO: Total 9000 docs.


In [ ]:
logging.info('Start training...')
from gensim.models.word2vec import Word2Vec

num_features = 100     # Word vector dimensionality
num_workers = 8       # Number of threads to run in parallel

train_texts = list(map(lambda x: list(x.split()), train_texts))
model = Word2Vec(train_texts, workers=num_workers, size=num_features)
model.init_sims(replace=True)

# save model
model.save("./word2vec.bin")

2021-08-12 16:47:55,289 INFO: Start training...
c:\zhangli\software\installer\python38\lib\site-packages\gensim\utils.py:1212: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")
2021-08-12 16:47:59,560 INFO: 'pattern' package not found; tag filters are not available for English
c:\zhangli\software\installer\python38\lib\site-packages\gensim\models\base_any2vec.py:742: UserWarning: C extension not loaded, training will be slow. Install a C compiler and reinstall gensim for fast training.
  warnings.warn(
2021-08-12 16:48:00,342 INFO: collecting all words and their counts
2021-08-12 16:48:00,349 INFO: PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2021-08-12 16:48:01,938 INFO: collected 5295 word types from a corpus of 8191447 raw words and 9000 sentences
2021-08-12 16:48:01,939 INFO: Loading a fresh vocabulary
2021-08-12 16:48:02,023 INFO: effective_min_count=5 retains 4335 uniq